In [209]:
import subprocess
import os

def run_git_commands(commands):
    """
    Executes the provided commands sequentially.
    """
    final_command = " && ".join(commands)
    try:
        # For mkdir and git init, use the shell=True option to support commands like `mkdir` and `cd`
        result = subprocess.run(final_command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        return result
    except subprocess.CalledProcessError as e:
        return type('result', (object,), {'stdout': e.stdout, 'stderr': e.stderr})

In [210]:
def git_init_commands():
    """
    Fill in your commands to create a git repository.
    """
    
    mkdir = "mkdir ~/git-workflow"
    cd = "cd ~/git-workflow"
    git_init = "git init"

    return mkdir, cd, git_init

In [211]:
import os
import shutil

repo_dir = os.path.expanduser("~/git-workflow")

# Clean up any previous test runs
if os.path.exists(repo_dir):
    shutil.rmtree(repo_dir)

# Get the student's commands
mkdir_cmd, cd_cmd, git_init_cmd = git_init_commands()

# Run the student's mkdir commands
run_git_commands([mkdir_cmd, cd_cmd, git_init_cmd])

# Check if the .git directory exists, which indicates a git repo was created
try:
    assert(os.path.isdir(os.path.join(repo_dir, ".git")))
except AssertionError as e:
    # Clean up the test directory
    if os.path.exists(repo_dir):
        shutil.rmtree(repo_dir)
    print(f"Error: {e}")


In [212]:
def check_untracked_files():
    """
    Fill in the git status command after creating the README.md and my_abs.py files
    """

    git_status = "git status"

    return git_status

In [213]:
# Create files if not existing in directory yet
files_and_contents = [
    ("README.md", "# Git Workflow\n\nThis is an example repo for the GT OSPO VSIP Summer 2024 Program."),
    ("my_abs.py", "def my_abs(x):\n    if x < 0:\n        return -x\n    else:\n        return x\n")
]

for file_name, content in files_and_contents:
    file_path = os.path.join(repo_dir, file_name)
    if not os.path.exists(file_path):
        with open(file_path, 'w') as file:
            file.write(content)  # Create an empty file

git_status_cmd = check_untracked_files()
result = run_git_commands([cd_cmd, git_status_cmd])

expected_output = 'No commits yet\n\nUntracked files:\n  (use "git add <file>..." to include in what will be committed)\n\tREADME.md\n\tmy_abs.py'

assert(expected_output in result.stdout)

In [214]:
def git_add_commands():
    """
    Fill in the git add command and git status after making all the changes and fixing README.md
    """
    
    # add both README.md and my_abs.py
    git_add = "git add README.md my_abs.py"
    git_status = "git status"
    
    return git_add, git_status

In [215]:
git_add_cmd, git_status_cmd = git_add_commands()
result = run_git_commands([cd_cmd, git_add_cmd, git_status_cmd])

expected_output = 'No commits yet\n\nChanges to be committed:\n  (use "git rm --cached <file>..." to unstage)\n\tnew file:   README.md\n\tnew file:   my_abs.py'

assert(expected_output in result.stdout)

In [216]:
def git_commit_command():
    """
    Fill in the git commit command and add the message
    """

    git_commit = 'git commit -m "First commit of my_abs (no try/except yet)"'

    return git_commit

In [217]:
git_commit_cmd = git_commit_command()
result = run_git_commands([cd_cmd, git_commit_cmd])

assert("2 files changed" in result.stdout and "\n create mode 100644 README.md\n create mode 100644 my_abs.py" in result.stdout)

In [218]:
# Insert your repo URL here for the unit test
repo_url = "git@github.com:natekim27/git-workflow.git"

def git_remote_command():
    """
    Fill in the git remote command after creating your repo
    """

    git_remote = "git remote add origin git@github.com:natekim27/git-workflow.git"
    git_remote_v = "git remote -v"

    return git_remote, git_remote_v

In [219]:
git_remote_cmd, git_remote_v_cmd = git_remote_command()
result = run_git_commands([cd_cmd, git_remote_cmd, git_remote_v_cmd])

expected_output = "origin\t" + repo_url + " (fetch)\norigin\t" + repo_url + " (push)"
assert(expected_output in result.stdout)

In [220]:
def git_push_command():
    """
    Fill in this git push command
    """

    git_push = "git push -u origin master"

    return git_push

In [221]:
git_push_cmd = git_push_command()
result = run_git_commands([cd_cmd, git_push_cmd])

expected_outputs = [
    "Branch 'master' set up to track remote branch 'master' from 'origin'.",
    "Updates were rejected because the remote contains work that you do\nhint: not have locally."
]

test_passed = any(expected_output in result.stdout or expected_output in result.stderr for expected_output in expected_outputs)

assert test_passed, "Test failed: None of the expected outputs were found."

In [222]:
def git_clone_commands():
    """
    Fill in the commands to make a new directory, cd into it, and clone the repo
    """
    mkdir_cmd = "mkdir ~/somewhere-else"
    cd_cmd = "cd ~/somewhere-else"
    git_clone_cmd = "git clone git@github.com:natekim27/git-workflow.git"

    return mkdir_cmd, cd_cmd, git_clone_cmd

In [223]:
repo_dir = os.path.expanduser("~/somewhere-else")

# Clean up any previous test runs
if os.path.exists(repo_dir):
    shutil.rmtree(repo_dir)

mkdir_cmd, cd_cmd, git_clone_cmd = git_clone_commands()
result = run_git_commands([mkdir_cmd, cd_cmd, git_clone_cmd])

expected_output = "Cloning into 'git-workflow'...\n"
assert(expected_output in result.stderr)

In [224]:
def git_diff_commands():
    """
    Fill in the command to cd into the original working copy and to run git diff after making changes to my_abs.py
    """
    git_workflow_cd_cmd = "cd ~/git-workflow"
    git_diff_cmd = "git diff"

    return git_workflow_cd_cmd, git_diff_cmd

In [225]:
expected_content = """import math

def my_abs(x):
    try:
        if x < 0:
            return -x
        else:
            return x
    except TypeError:
        return math.nan
"""
file_path = os.path.expanduser("~/git-workflow/my_abs.py")
current_content = ''
if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        current_content = file.read()

if current_content != expected_content:
    with open(file_path, 'w') as file:
        file.write(expected_content)

cd_cmd, git_diff_cmd = git_diff_commands()
result = run_git_commands([cd_cmd, git_diff_cmd])
expected_results = [
    "diff --git a/my_abs.py b/my_abs.py",
    """+import math
+
 def my_abs(x):
-    if x < 0:
-        return -x
-    else:
-        return x""",
    """+    try:
+        if x < 0:
+            return -x
+        else:
+            return x
+    except TypeError:
+        return math.nan"""
]

assert(all(expected_result in result.stdout for expected_result in expected_results))

In [226]:
def git_commit_am_command():
    """
    Fill in the git commit command with the -a flag and add the message.
    Fill in the git push command
    """

    git_commit_am = 'git commit -am "Added try/except"'
    git_push_cmd = "git push"
    
    return git_commit_am, git_push_cmd

In [228]:
import re

git_commit_am, git_push_cmd = git_commit_am_command()

result = run_git_commands([cd_cmd, git_commit_am, git_push_cmd])
expected_outputs = [
    r"To github.com:.*/git-workflow.git\n\s+\w{7}\.\.\w{7}\s+master\s+->\s+master",
    "Your branch is up to date with 'origin/master'.\n\nnothing to commit, working tree clean"
]

test_passed = any(re.search(expected_output, result.stdout + result.stderr) for expected_output in expected_outputs)

assert test_passed, "Test failed: None of the expected outputs were found."